<a href="https://colab.research.google.com/github/FireStrings/MasterDegree/blob/main/LSTM/Old/LSTM_BH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TODO
- Mutiplicar os valores por 1000 no resultado.
- (FICOU UMA BOSTA) agrupar o plot do sarima para deixar a visualização mais adequada
- Coletar resultados do sarima e do LSTM
- Executar o BiLSTM e ver o resultado, e coletar tambem (colocar em uma planilha)
- Carregar o modelo LSTM e BiLSTM e testar em outras estações

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter, YearLocator
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError, R2Score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Dense, InputLayer, LSTM, Dropout, Conv1D, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, normalize, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import tensorflow as tf
import math

In [ ]:
%run ../UtilsNew.ipynb

### Load and filter data

In [ ]:
raw_anual_df1 = pd.read_csv("../data/minas_gerais_bh.csv", sep=";")
raw_anual_df2 = pd.read_csv("../data/minas_gerais_bh2.csv", sep=";")

raw_anual_df = pd.concat([raw_anual_df1, raw_anual_df2], axis=0)

anual_df = pre_processing(raw_anual_df)

anual_df["hora"] = anual_df["data_hora"].dt.hour
anual_df = filter_between(anual_df, "hora", 7, 18)

### Data segmentation

In [ ]:
train_size = int(0.7 * (round(len(anual_df)/10)*10))

train_df = anual_df.iloc[0:train_size]
test_df = anual_df.iloc[train_size:]

janela_tempo = 11

### Funções

In [ ]:
def normalize_leo(X):
    scaler = MinMaxScaler()
    return scaler.fit_transform(X)

def teste(values):
    scaler = StandardScaler()
    scaler = scaler.fit(values)
    # print('Mean: %f, StandardDeviation: %f' % (scaler.mean_, math.sqrt(scaler.var_)))
    # normalize the dataset and print
    standardized = scaler.transform(values)

    return standardized

def create_model(janela_tempo, with_bilstm):

    model = Sequential()
    model.add(InputLayer(input_shape=(janela_tempo, 1)))

    if with_bilstm:
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(32)))
    else:
        model.add(LSTM(64, return_sequences=True))  # Mantém a saída sequencial
        model.add(LSTM(32))

    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mse')

    return model

def train_model(model, X, y, epochs=100):
    model.fit(X, y, epochs=epochs, batch_size=32, verbose=0)

    return model

def plot_results(df_orig, pred, real):
    set_plot_size(12, 6)

    period = df_orig[0:len(df_orig) - janela_tempo]['data']
    pred = pred.reshape(pred.shape[0])

    df_pred = pd.DataFrame(data=pred, index=period, columns=["predicted"])
    df_real = pd.DataFrame(data=real, index=period, columns=["real"])

    sns.lineplot(df_pred, palette=["red"], ci=None)
    sns.lineplot(df_real, palette=["blue"], ci=None)


def get_metrics(test_pred, y_val):
    mse = mean_squared_error(test_pred.reshape(test_pred.shape[0]), y_val)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(test_pred.reshape(test_pred.shape[0]), y_val)
    r2 = r2_score(test_pred.reshape(test_pred.shape[0]), y_val)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2}

def data_to_input_and_output(data):
    input_data = []
    output_data = []
    for index in range(0, len(data) - janela_tempo):
        input_sample = data['radiacao'][index:index + janela_tempo]
        output_sample = data['radiacao'][index + janela_tempo]

        input_data.append(input_sample)
        output_data.append(output_sample)

    return np.array(input_data)/1000, np.array(output_data)/1000
    # return teste(np.array(input_data)), teste([np.array(output_data)])

def process(times, results, with_bilstm, with_plot):
    for i in range(times):
        print("Starting test " + str(i))
        results[i] = {"train_pred":None, "test_pred": None, "period": None, "metrics":None}

        model = create_model(janela_tempo, with_bilstm)
        train_model(model, train_input, train_output)

        train_pred = model.predict(train_input, verbose=0)
        test_pred = model.predict(test_input, verbose=0)

        if with_plot:
            period = anual_df[train_pred.shape[0]:]['data'].iloc[:test_pred.shape[0]]
            plot_results(test_pred.reshape(test_pred.shape[0]), test_output, period)

        results[i]["train_pred"] = train_pred
        results[i]["test_pred"] = test_pred
        results[i]["metrics"] = get_metrics(test_pred, test_output)

In [ ]:
train_input, train_output = data_to_input_and_output(train_df)
test_input, test_output = data_to_input_and_output(test_df)

## LSTM

In [ ]:
results_lstm = {}
process(15, results_lstm, False, False)

Starting test 0
82/82 [==============================] - 1s 6ms/step
Starting test 1
82/82 [==============================] - 1s 6ms/step
Starting test 2
82/82 [==============================] - 0s 5ms/step
Starting test 3
82/82 [==============================] - 0s 6ms/step
Starting test 4
82/82 [==============================] - 0s 6ms/step
Starting test 5
82/82 [==============================] - 0s 5ms/step
Starting test 6
82/82 [==============================] - 0s 6ms/step
Starting test 7
82/82 [==============================] - 0s 5ms/step
Starting test 8
82/82 [==============================] - 0s 5ms/step
Starting test 9
82/82 [==============================] - 0s 5ms/step
Starting test 10
82/82 [==============================] - 0s 5ms/step
Starting test 11
82/82 [==============================] - 0s 6ms/step
Starting test 12
82/82 [==============================] - 0s 6ms/step
Starting test 13
82/82 [==============================] - 1s 6ms/step
Starting test 14
82/82 [======

In [ ]:
for i in ["mse", "rmse", "mae", "r2"]:
    print(i, np.mean([results_lstm[x]["metrics"][i] for x in results_lstm]))

mse 0.14736095367026295
rmse 0.3838476365443627
mae 0.2573913759377284
r2 0.8506812632071286
